In [ ]:
import os
import re
import sys
import glob
import pickle
import tables
from collections import OrderedDict

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# import tensorflow as tf
# from tensorflow import keras
from comet_ml.api import API, APIExperiment
from comet_ml.query import Tag

if '..' not in sys.path:
    sys.path.append('..')
from deep_utils import *

In [ ]:
api = API(api_key = os.environ['COMET_API_KEY'])
workspace = 'danielelinaro'
project_name = 'inertia'
D = 2
DZA = 60
network_name = 'IEEE39'
inertia_units = 'GW s'
area_ID = 1
# inertia of generator 1
H_G1 = 500 # [s]
# the bus(es) where the stochastic load is connected
stoch_load_bus_IDs = [3]
stoch_load_bus_list = 'stoch_load_bus_' + '-'.join(map(str, stoch_load_bus_IDs))
# the bus(es) used for recording: an empy list means that the corresponding experiment tag won't be used
rec_bus_IDs = []

query = Tag(network_name) & \
        Tag(f'D={D}') & \
        Tag(f'DZA={DZA}') & \
        Tag('1D_pipeline') & \
        Tag(stoch_load_bus_list) & \
        Tag(f'H_G1_{H_G1}') & \
        Tag(f'area{area_ID}')

if len(rec_bus_IDs) > 1:
    rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))
    query &= Tag(rec_bus_list)

experiments = api.query(workspace, project_name, query, archived=False)
experiment_IDs = []
MAPE = []
val_loss = []
loss = []
batch_loss = []
tags =  []
for experiment in experiments:
    ID = experiment.id
    experiment_IDs.append(ID)
    sys.stdout.write(f'Downloading data for experiment ID {ID}... ')
    metrics = experiment.get_metrics()
    sys.stdout.write('done.\n')
    val_loss.append(np.array([float(m['metricValue']) for m in metrics if m['metricName'] == 'val_loss']))
    loss.append(np.array([float(m['metricValue']) for m in metrics if m['metricName'] == 'loss']))
    batch_loss.append(np.array([float(m['metricValue']) for m in metrics if m['metricName'] == 'batch_loss']))
    has_MAPE = False
    for m in metrics:
        if m['metricName'] == 'mape_prediction':
            val = m['metricValue']
            try:
                MAPE.append(float(val))
            except:
                MAPE.append(list(map(float, [v for v in val[1:-1].split(' ') if len(v)])))
            has_MAPE = True
            break
    tags.append(experiment.get_tags())
    print(f'  val_loss: {val_loss[-1].min():.4f}')
    if has_MAPE:
        print(f'      MAPE: {MAPE[-1]}%')
    else:
        print('      MAPE: [experiment not terminated]')
    print('      Tags: "{}"'.format('" "'.join(tags[-1])))
idx = np.argmin([loss.min() for loss in val_loss])
experiment_ID = experiment_IDs[idx]
MAPE = MAPE[idx]
val_loss = val_loss[idx]
loss = loss[idx]
batch_loss = batch_loss[idx]
tags = tags[idx]
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE}%).')

experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(experiments_path + experiment_ID + '/parameters.pkl', 'rb'))
test_results = pickle.load(open(experiments_path + experiment_ID + '/test_results.pkl', 'rb'))

In [ ]:
n_epochs = len(loss)
epochs = np.arange(n_epochs) + 1

font = {'family' : 'Times New Roman',
        'size'   : 8}
matplotlib.rc('font', **font)

fig,ax = plt.subplots(1, 2, figsize=(3.3, 1.8))
ax[0].semilogy(epochs, val_loss, color=[.8,.8,.8], lw=1, label='Validation set')
ax[0].semilogy(epochs, loss, 'k', lw=1, label='Training set')
ax[0].legend(loc='upper right', fontsize=7)
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('MAE')
ax[0].set_xticks(np.r_[0 : 650 : 150])
y_test, y_prediction = test_results['y_test'], test_results['y_prediction']
ax[1].plot([8,14.5], [8,14.5], lw=2, color=[.8,.8,.8])
ax[1].plot(y_test, y_prediction, 'o', color=[.6,.6,.6], markerfacecolor='w', markersize=3, markeredgewidth=0.5)
for x in np.unique(y_test):
    idx, = np.where(y_test == x)
    ymean, ystd = y_prediction[idx].mean(), y_prediction[idx].std()
    ysem = ystd / np.sqrt(len(idx))
    ax[1].plot(x + np.zeros(2), ymean + 3 * ystd * np.array([-1,1]), 'k', linewidth=1)
    ax[1].plot(x, ymean, 'ko', markerfacecolor='w', markersize=3.75, markeredgewidth=1)
ax[1].set_xlabel(r'Inertia [GW$\cdot$s]')
ax[1].set_ylabel(r'Predicted inertia [GW$\cdot$s]')
ax[1].set_xticks(np.r_[8:16])
ax[1].set_yticks(np.r_[8:16])
ax[1].text(8.5, 14.5, f'MAPE = {MAPE:.2f}%', fontsize=7)

for a in ax:
    for side in 'top','right':
        a.spines[side].set_visible(False)

fig.tight_layout(pad=0.1)
fig.savefig(f'training_results_{experiment_ID[:6]}.pdf')